In [ ]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import sys
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from tqdm import tqdm
# Load image
import imageio


# Set random seed so output is all same
np.random.seed(1)


class KMeans(object):

    def __init__(self):  # No need to implement
        pass

    def pairwise_dist(self, x, y):  # [5 pts]
        """
        Args:
            x: N x D numpy array
            y: M x D numpy array
        Return:
                dist: N x M array, where dist2[i, j] is the euclidean distance between 
                x[i, :] and y[j, :]
                """
        xSumSquare = np.sum(np.square(x),axis=1);
        ySumSquare = np.sum(np.square(y),axis=1);
        mul = np.dot(x, y.T);
        dists = np.sqrt(abs(xSumSquare[:, np.newaxis] + ySumSquare-2*mul))
        return dists

    def _init_centers(self, points, K, **kwargs):  # [5 pts]
        """
        Args:
            points: NxD numpy array, where N is # points and D is the dimensionality
            K: number of clusters
            kwargs: any additional arguments you want
        Return:
            centers: K x D numpy array, the centers. 
        """
        ## Here initial points are chosen at random
        row, col = points.shape
        retArr = np.empty([K, col])
        for number in range(K):
            randIndex = np.random.randint(row)
            retArr[number] = points[randIndex]
        print(retArr)
        ## I have assigned intial points as first 2 points here(as asked in the assigmnent question) but you can change this according to the question in exam
        retArr = points[0:2]
        return retArr

    def _update_assignment(self, centers, points):  # [10 pts]
        """
        Args:
            centers: KxD numpy array, where K is the number of clusters, and D is the dimension
            points: NxD numpy array, the observations
        Return:
            cluster_idx: numpy array of length N, the cluster assignment for each point

        Hint: You could call pairwise_dist() function.
        """
        row, col = points.shape
        cluster_idx = np.empty([row])
        distances = self.pairwise_dist(points, centers)
        print("dist matrix:")
        print(distances)
        cluster_idx = np.argmin(distances, axis=1)

        return cluster_idx

    def _update_centers(self, old_centers, cluster_idx, points):  # [10 pts]
        """
        Args:
            old_centers: old centers KxD numpy array, where K is the number of clusters, and D is the dimension
            cluster_idx: numpy array of length N, the cluster assignment for each point
            points: NxD numpy array, the observations
        Return:
            centers: new centers, K x D numpy array, where K is the number of clusters, and D is the dimension.
        """
        K, D = old_centers.shape
        new_centers = np.empty(old_centers.shape)
        for i in range(K):
            new_centers[i] = np.mean(points[cluster_idx == i], axis = 0)
        print("new_centers: ")
        print(new_centers)
        return new_centers

    def _get_loss(self, centers, cluster_idx, points):  # [5 pts]
        """
        Args:
            centers: KxD numpy array, where K is the number of clusters, and D is the dimension
            cluster_idx: numpy array of length N, the cluster assignment for each point
            points: NxD numpy array, the observations
        Return:
            loss: a single float number, which is the objective function of KMeans. 
        """
        dists = self.pairwise_dist(points, centers)
        print("dist in calculating loss :")
        print(dists)
        loss = 0.0
        N, D = points.shape
        for i in range(N):
            print(dists[i][cluster_idx[i]])
            loss = loss + np.square(dists[i][cluster_idx[i]])
        print("loss:",loss)
        return loss

    def __call__(self, points, K, max_iters=100, abs_tol=1e-16, rel_tol=1e-16, verbose=False, **kwargs):
        """
        Args:
            points: NxD numpy array, where N is # points and D is the dimensionality
            K: number of clusters
            max_iters: maximum number of iterations (Hint: You could change it when debugging)
            abs_tol: convergence criteria w.r.t absolute change of loss
            rel_tol: convergence criteria w.r.t relative change of loss
            verbose: boolean to set whether method should print loss (Hint: helpful for debugging)
            kwargs: any additional arguments you want
        Return:
            cluster assignments: Nx1 int numpy array
            cluster centers: K x D numpy array, the centers
            loss: final loss value of the objective function of KMeans
        """
        centers = self._init_centers(points, K, **kwargs)
        for it in range(max_iters):
            cluster_idx = self._update_assignment(centers, points)
  
            loss = self._get_loss(centers, cluster_idx, points)
            centers = self._update_centers(centers, cluster_idx, points)    
            K = centers.shape[0]
            if it:
                diff = np.abs(prev_loss - loss)
                if diff < abs_tol and diff / prev_loss < rel_tol:
                    break
            prev_loss = loss
            if verbose:
                print('iter %d, loss: %.4f' % (it, loss))
        return cluster_idx, centers, loss

In [ ]:
k = 2 # feel free to change this value
X = np.array([[0, 2],
              [0, 0],
              [2, 0 ],
              [5, 0],
              [5, 2]])
cluster_idx, centers, loss = KMeans()(X, k)
updated_image_values = np.copy(X)

[[5. 0.]
 [5. 2.]]
dist matrix:
[[0.         2.        ]
 [2.         0.        ]
 [2.82842712 2.        ]
 [5.38516481 5.        ]
 [5.         5.38516481]]
dist in calculating loss :
[[0.         2.        ]
 [2.         0.        ]
 [2.82842712 2.        ]
 [5.38516481 5.        ]
 [5.         5.38516481]]
0.0
0.0
2.0
5.0
5.0
loss: 54.0
new_centers: 
[[2.5        2.        ]
 [2.33333333 0.        ]]
dist matrix:
[[2.5        3.07318149]
 [3.20156212 2.33333333]
 [2.06155281 0.33333333]
 [3.20156212 2.66666667]
 [2.5        3.33333333]]
dist in calculating loss :
[[2.5        3.07318149]
 [3.20156212 2.33333333]
 [2.06155281 0.33333333]
 [3.20156212 2.66666667]
 [2.5        3.33333333]]
2.5
2.3333333333333335
0.3333333333333354
2.6666666666666665
2.5
loss: 25.16666666666667
new_centers: 
[[2.5        2.        ]
 [2.33333333 0.        ]]
dist matrix:
[[2.5        3.07318149]
 [3.20156212 2.33333333]
 [2.06155281 0.33333333]
 [3.20156212 2.66666667]
 [2.5        3.33333333]]
dist in 

In [ ]:
print(cluster_idx)
print(centers)
print(loss)

[0 1 1 1 0]
[[2.5        2.        ]
 [2.33333333 0.        ]]
25.16666666666667
